In [2]:
from datetime import date
from os import mkdir, makedirs
from tqdm import tqdm
import pandas as pd
RNASEQ_PIPELINE_RESULTS_PATH = "../RNASeq_pipeline/Results" #-%s" % date.today().strftime("%Y%m%d")
DATAPREP_RESULTS_PATH = "Results"#-%s" % date.today().strftime("%Y%m%d")
makedirs(DATAPREP_RESULTS_PATH, exist_ok=True)  # adding this from original code to avoid directory issues

In [ ]:
import itertools
import numpy as np
for region in ["BLA", "CPU", "NAC", "PFC", "HIP", "VTA"]:
    Voom_DEG_df = pd.read_csv(RNASEQ_PIPELINE_RESULTS_PATH + "/Voom_Result_%s.csv" % region,index_col = 0)
    groups = {}
    Voom_DEG_df_aug = pd.DataFrame(index=Voom_DEG_df.index)

    for col in Voom_DEG_df.columns:
        if col[9:11] not in groups.keys():
            groups[col[9:11]] = []
        groups[col[9:11]].append(col)
    com_groups = {}

    for key in groups.keys():
        com_groups[key] = list(map(','.join, itertools.combinations(groups[key], 2)))
    com_groups

    for key in com_groups.keys():
        for com in com_groups[key]:
            for x in np.arange(0.1, 1.0, 0.1):
                col_name = ""
                col_name += "[" + region + "]"
                col_name += "[" + key + "]"
                col_name += "[" + com.split(",")[0][1:4] + "-" + com.split(",")[1][1:4] + "]"
                col_name += "[" + str(x)[0:3] + "]"
                ai1 = float(com.split(",")[0][12:])
                ai2 = float(com.split(",")[1][12:])
                if (com.split(",")[0][12:] == "0.0" or com.split(",")[1][12:] == "0.0"):
                    col_name += "[None]"
                else:
                    ai_aug = str((x * ai1) + ((1.0 - x) * ai2))
                    col_name += "[" + ai_aug + "]"
                c = (x * Voom_DEG_df[com.split(",")[0]].values) + ((1.0 - x) * Voom_DEG_df[com.split(",")[1]].values)
                Voom_DEG_df_aug[col_name] = c # / 10.0

    for col in Voom_DEG_df.columns:
        col_name = ""
        col_name += "[" + col[5:8] + "]"
        col_name += "[" + col[9:11] + "]"
        col_name += "[" + col[1:4] + "]"
        col_name += "[1.0]"
        col_name += "[" + col[12:] + "]"
        Voom_DEG_df_aug[col_name] = Voom_DEG_df[col]
                
    Voom_DEG_df_aug.to_csv(DATAPREP_RESULTS_PATH + "/%s_Voom_Augmented_Result.csv" % region)